In [32]:
import sqlite3
import pandas as pd
from tools import find_tables_info
import plotly.express as px

# Connect to SQLite database
conn = sqlite3.connect(r'./db/vivino.db')
  
# Create a cursor object
cur = conn.cursor()

In [3]:
q = """
SELECT name
FROM sqlite_schema
WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
"""

cur.execute(q).fetchall()

[('countries',),
 ('grapes',),
 ('wineries',),
 ('flavor_groups',),
 ('keywords',),
 ('regions',),
 ('most_used_grapes_per_country',),
 ('toplists',),
 ('wines',),
 ('vintages',),
 ('keywords_wine',),
 ('vintage_toplists_rankings',)]

# We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

In [4]:
q = """
SELECT name, price_euros, ratings_average
FROM (
SELECT *
FROM vintages
JOIN wines
ON wines.id = vintages.wine_id
ORDER BY ratings_average DESC
LIMIT 10);
"""

cur.execute(q).fetchall()
df  = pd.read_sql_query (q, conn)
df

,name,price_euros,ratings_average
0,Château Doisy-Daëne L'Extravagant de Doisy-Daë...,397.50,4.9
1,Biondi-Santi Brunello di Montalcino Riserva 1955,1970.05,4.9
2,Château Lafite Rothschild Pauillac (Premier Gr...,1900.00,4.8
3,Krug Clos du Mesnil 1998,3522.50,4.8
4,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...,2882.50,4.8
5,Dal Forno Romano Amarone della Valpolicella Mo...,808.75,4.8
6,Harlan Estate Harlan Estate Red 1998,1921.25,4.8
7,Krug Clos du Mesnil 2000,3096.25,4.8
8,Quinta do Noval Vintage Port Nacional 2001,1152.50,4.8
9,Pétrus Pomerol 1990,6300.00,4.8


In [5]:
q = """

SELECT wines.name, user_structure_count
FROM vintages
JOIN wines
ON wines.id = vintages.wine_id
GROUP BY wines.name
ORDER BY user_structure_count DESC
LIMIT 10;
"""

cur.execute(q).fetchall()
df  = pd.read_sql_query (q, conn)
df

,name,user_structure_count
0,Brut Champagne,13491
1,Tignanello,8219
2,Malbec Argentino,5360
3,Grande Cuvée,5360
4,Cristal Brut Champagne (Millésimé),5101
5,Valbuena 5º,4638
6,Ribera del Duero,4511
7,Bramare Malbec Uco Valley,3810
8,Malbec,3716
9,Comtes de Champagne Blanc de Blancs,3656


In [6]:
q = """
WITH q AS (
    SELECT *
    FROM toplists
    JOIN countries ON toplists.country_code = countries.code
),
p AS (
    SELECT *
    FROM vintages
    JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
)
    SELECT DISTINCT p.name, p.price_euros, q.name, p.wine_id
    FROM q
    JOIN p ON p.top_list_id = q.id
    ORDER BY q.users_count DESC
    LIMIT 10;

"""

cur.execute(q).fetchall()
df  = pd.read_sql_query (q, conn)

df

,name,price_euros,name,wine_id
0,Gitana Lupi Rezerva 2017,36.55,Bestsellers in Arizona,1468452
1,Dom Pérignon Brut Champagne 2013,250.00,Bestsellers in Tennessee,86684
2,Billecart-Salmon Brut Rosé Champagne,81.50,Bestsellers in Arizona,1211816
3,Buena Vista Chateau Buena Vista Cabernet Sauvi...,61.95,Bestsellers in Utah,5261
4,Zenato Amarone della Valpolicella Classico 2017,51.90,Bestsellers in Nevada,12393
5,San Marzano 60 Sessantanni Old Vines Primitivo...,24.75,Bestsellers in Massachusetts,11890
6,San Marzano 60 Sessantanni Old Vines Primitivo...,24.75,Bestsellers in North Dakota,11890
7,Chateau Montelena Chardonnay 2019,78.95,Bestsellers in Montana,1697
8,Château Pape Clément Pessac-Léognan (Grand Cru...,121.95,Bestsellers in New Mexico,82613
9,Kistler Les Noisetiers 2020,94.00,Bestsellers in Wyoming,1155762


# We have a limited marketing budget for this year. Which country should we prioritise and why?


In [7]:
q = """ 
SELECT name, users_count
FROM countries
ORDER by users_count DESC
LIMIT 1
"""

cur.execute(q).fetchall()

df  = pd.read_sql_query (q, conn)
df


,name,users_count
0,États-Unis,12273684


# We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?


## Most wines produced

In [8]:
q = """ 
WITH wineries_ids AS (
SELECT name AS wine_name, winery_id, count(*) AS num_wines, id AS wine_id
FROM wines
GROUP BY winery_id)

SELECT SUBSTR(REPLACE(vintages.name, wineries_ids.wine_name, ''), 1, LENGTH(REPLACE(vintages.name, wineries_ids.wine_name, ''))-5) as winery_name, 
wineries_ids.wine_name, wineries_ids.winery_id, wineries_ids.num_wines
FROM vintages
JOIN wineries_ids ON wineries_ids.wine_id = vintages.wine_id
GROUP BY winery_name
ORDER BY num_wines DESC
LIMIT 3



"""
cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df



,winery_name,wine_name,winery_id,num_wines
0,Gaja,Barbaresco,10474,12
1,Domaine Faiveley,Nuits-Saint-Georges 1er Cru Les Damodes,9294,10
2,Antinori,Tenuta Tignanello 'Solaia',1252,9


## Quantity of the bestsellers produced


In [9]:
rq = """
WITH q AS (
    SELECT COUNT(*) AS num_wines, vintages.name as winery_name, wines.name as wine_name, wines.winery_id
    FROM toplists
    JOIN vintage_toplists_rankings ON vintage_toplists_rankings.top_list_id = toplists.id
    JOIN vintages ON vintage_toplists_rankings.vintage_id = vintages.id
    JOIN wines ON vintages.wine_id = wines.id 
    GROUP BY wines.winery_id 
)
SELECT SUBSTR(REPLACE(winery_name, wine_name, ''), 1, LENGTH(REPLACE(winery_name, wine_name, ''))-5) as winery_name, 
wine_name, winery_id, num_wines
FROM q
ORDER BY num_wines DESC
LIMIT 3



"""

cur.execute(rq).fetchall()
df  = pd.read_sql_query (rq, conn)
df

#these wineryids also doesn't appear in wineries table 


,winery_name,wine_name,winery_id,num_wines
0,Antinori,Tenuta Tignanello 'Solaia',1252,10
1,Krug,Clos du Mesnil,1337,8
2,Dal Forno Romano,Amarone della Valpolicella Monte Lodoletta,1852,6


## Highest average rating

In [10]:
q = """ 
WITH q AS (
SELECT winery_id, AVG(ratings_average) AS avg_rating
FROM wines
GROUP BY winery_id),

wineries AS (
SELECT SUBSTR(REPLACE(vintages.name, wines.name, ''), 1, LENGTH(REPLACE(vintages.name, wines.name, ''))-5) as winery_name, 
wines.winery_id
FROM vintages
JOIN wines ON vintages.wine_id = wines.id
GROUP BY winery_name)

SELECT wineries.winery_name, q.winery_id, q.avg_rating
FROM q
JOIN wineries ON wineries.winery_id = q.winery_id
ORDER BY avg_rating DESC
LIMIT 3


"""
cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


,winery_name,winery_id,avg_rating
0,Scarecrow,14919,4.8
1,Pierre Girardin,266660,4.8
2,Tenuta Hortense,277785,4.8


# We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

# All the wines that have the combination of all the tastes

In [46]:

q = """ 
WITH flavour_ids AS (
    SELECT *
    FROM keywords
    WHERE name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
), needed_wines AS (
SELECT wine_id, COUNT(DISTINCT keyword_id) AS unique_keywords
    FROM keywords_wine 
    WHERE keyword_id IN (
        SELECT id
        FROM flavour_ids
    ) AND count > 10
    GROUP BY wine_id
    HAVING unique_keywords = 5)

SELECT group_name, count as num, wine_id 
FROM keywords_wine
WHERE wine_id IN
(SELECT wine_id 
FROM needed_wines)



"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


for wine_id in df['wine_id'].unique():
    #  Filter DataFrame for current wine ID
    filtered_df = df[df['wine_id'] == wine_id]
    
    # Create line polar graph
    fig = px.line_polar(filtered_df, r='num', theta='group_name', line_close=True, title=f'Wine ID: {wine_id}')
    fig.show()


# We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.

In [13]:
q = """ 
WITH grape_ids AS (
SELECT grape_id, SUM(wines_count) as total_wine, country_code
FROM most_used_grapes_per_country
GROUP BY grape_id
ORDER BY total_wine DESC
LIMIT 3),
grape_name AS(
SELECT name, id, country_code
FROM grapes
JOIN grape_ids ON grape_ids.grape_id = grapes.id),
rating AS (
SELECT wines.name, grape_name.name AS grape_name, wines.ratings_average 
FROM wines 
JOIN regions ON wines.region_id = regions.id
JOIN grape_name ON grape_name.country_code = regions.country_code
ORDER BY wines.ratings_average DESC),
cabernet AS (
SELECT *
FROM rating 
WHERE grape_name = 'Cabernet Sauvignon'
LIMIT 5),
merlot AS (
SELECT *
FROM rating 
WHERE grape_name = 'Merlot'
LIMIT 5)

SELECT *
FROM rating 
WHERE grape_name = 'Chardonnay'
LIMIT 5

"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df

,name,grape_name,ratings_average
0,Amarone della Valpolicella Classico Riserva,Chardonnay,4.8
1,Fratini Bolgheri Superiore,Chardonnay,4.8
2,Cerretalto Brunello di Montalcino,Chardonnay,4.7
3,Barolo Riserva Monfortino,Chardonnay,4.7
4,Amarone della Valpolicella Classico,Chardonnay,4.7


# We would like to create a country leaderboard. Come up with a visual that shows the average wine rating for each country. Do the same for the vintages.

In [14]:
q = """ 
SELECT name, wines_count
FROM countries
"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)

import plotly.express as px
fig = px.line(df, x='name', y="wines_count",
               labels={
                     "name": "Countries",
                     "wines_count": "Wines Count"})
fig.show()

In [15]:
q = """ 
SELECT region_id, AVG(ratings_average) as average_rating, countries.name
FROM wines
JOIN regions ON regions.id = wines.region_id
JOIN countries ON regions.country_code = countries.code
GROUP BY countries.name


"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df
import plotly.express as px
fig = px.line(df, x='name', y="average_rating",
               labels={
                     "name": "Countries",
                     "wines_count": "Wines Rating"})
fig.show()

In [16]:
q = """ 
SELECT wines.region_id, AVG(vintages.ratings_average) as average_rating, countries.name
FROM vintages
JOIN wines ON vintages.wine_id = wines.id
JOIN regions ON regions.id = wines.region_id
JOIN countries ON regions.country_code = countries.code
WHERE vintages.ratings_average <> 0
GROUP BY countries.name


"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df
import plotly.express as px
fig = px.line(df, x='name', y="average_rating",
               labels={
                     "name": "Countries",
                     "wines_count": "Wines Rating"})
fig.show()

# One of our VIP clients likes _Cabernet Sauvignon_ and would like our top 5 recommendations. Which wines would you recommend to him?


In [17]:
q = """ 
WITH w AS (
    SELECT
        name,
        id,
        AVG(acidity) AS avg_acidity,
        AVG(fizziness) AS avg_fizziness,
        AVG(intensity) AS avg_intensity,
        AVG(sweetness) AS avg_sweetness,
        AVG(tannin) AS avg_tannin
    FROM wines
    WHERE acidity IS NOT NULL AND intensity IS NOT NULL 
          AND sweetness IS NOT NULL AND tannin IS NOT NULL
    GROUP BY name
), 
a AS (
    SELECT         
        AVG(acidity) AS avg_acidity,
        AVG(fizziness) AS avg_fizziness,
        AVG(intensity) AS avg_intensity,
        AVG(sweetness) AS avg_sweetness,
        AVG(tannin) AS avg_tannin
    FROM wines
    WHERE name = 'Cabernet Sauvignon'
), top_wines AS (
SELECT w.name,
       w.id,
       w.avg_acidity,
       w.avg_fizziness,
       w.avg_intensity,
       w.avg_sweetness,
       w.avg_tannin,
       ABS(w.avg_acidity - a.avg_acidity) as acidity_difference,
       ABS(w.avg_intensity - a.avg_intensity) as intensity_difference,
       ABS(w.avg_sweetness - a.avg_sweetness) as sweetness_difference,
       ABS(w.avg_tannin - a.avg_tannin) as tannin_difference,
       (ABS(w.avg_acidity - a.avg_acidity) *  ABS(w.avg_intensity - a.avg_intensity) * ABS(w.avg_sweetness - a.avg_sweetness) * ABS(w.avg_tannin - a.avg_tannin)) AS combined_difference
       
FROM w
JOIN a
WHERE name NOT LIKE '%Cabernet Sauvignon%' 
ORDER BY combined_difference
LIMIT 5
)
SELECT *
FROM top_wines



"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


,name,id,avg_acidity,avg_fizziness,avg_intensity,avg_sweetness,avg_tannin,acidity_difference,intensity_difference,sweetness_difference,tannin_difference,combined_difference
0,Altura,18323,3.088019,None,4.335320,1.750534,3.434145,0.223193,0.060686,0.002190,0.015943,4.729554e-07
1,Montelìg,1131354,3.112812,None,4.594468,1.748861,3.491792,0.198399,0.198462,0.000517,0.041703,8.490276e-07
2,Victorino,1182681,2.778029,None,4.398544,1.834238,3.422341,0.533183,0.002538,0.085894,0.027747,3.224923e-06
3,Pomerol,18555,3.404267,None,4.160957,1.736402,3.471022,0.093056,0.235049,0.011942,0.020934,5.468098e-06
4,Torre Muga,7494,3.779983,None,3.976726,1.748448,3.914909,0.468771,0.419280,0.000104,0.464821,9.469336e-06


In [29]:
q = """ 
SELECT count(wines.name) AS num_wines, region_id, regions.name, countries.name as country
FROM wines
JOIN regions ON regions.id = wines.region_id
JOIN countries ON regions.country_code = countries.code
GROUP BY region_id
ORDER BY num_wines DESC
LIMIT 10
"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


import plotly.express as px
fig = px.line(df, x='name', y="num_wines",
               labels={
                     "name": "Regions",
                     "num_wines": "Wines Count"})
fig.show()